In [1]:
from IPython.display import display
from fastprogress import master_bar, progress_bar
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=0.9)
sns.set_style("whitegrid")
sns.set_style({'font.family':'monospace'})
import os
import ntpath
from datetime import datetime
import numpy as np
import statistics 
import re
import math
import random
import pickle
import scipy.stats as scstat            
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
from unidecode import unidecode
import datetime
from datetime import timedelta
import numbers
from collections.abc import MutableMapping
from fastprogress import master_bar, progress_bar
import requests,json,psutil,datetime
import timeit

import sklearn.model_selection as model_selection
import sklearn.metrics as metrics

# If true exports vectorial PDFs instead of JPG.
VECTORIAL_FIGURES = False
FIG_EXTENSION = "pdf" if VECTORIAL_FIGURES else "jpg"

ROOT_DIR = "/home/mattia/DGA-Test/"
DATA_DIR = ROOT_DIR + "Data/"
MODELS_DIR = ROOT_DIR + "Models/"
GRAPHICS_DIR = ROOT_DIR + "Graphics/" + FIG_EXTENSION + "/"

# Change path to root
os.chdir(ROOT_DIR)
print(os.getcwd())

try:
    os.makedirs(GRAPHICS_DIR)
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(MODELS_DIR)
except FileExistsError:
    # directory already exists
    pass

import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
from scipy.io import arff
from resource import getrusage as resource_usage, RUSAGE_SELF
from time import time as timestamp

/home/mattia/DGA-Test


## Preprocessing

In [2]:
to_be_replaced = {
    'kraken': ['kraken_v1', 'kraken_v2'],
    'CL01': ['alureon', 'fobber_v2'],
    'CL02': ['gozi_gpl','gozi_luther', 'gozi_nasa', 'gozi_rfc4343', 'rovnix'],
    'CL03': ['pykspa_noise', 'pykspa','proslikefan', 'tempedreve', 'qadars'],
    'CL04': ['vawtrak_v2', 'vawtrak_v3'],
    'CL05': ['pizd', 'suppobox_1'],
    'CL06': ['dircrypt', 'bedep', 'ramnit'],
    'CL07': ['ranbyus_v1', 'fobber_v1', 'cryptolocker'],
    'CL08': ['ranbyus_v2', 'murofet_v2'],
    'CL09': ['qakbot', 'murofet_v1'],
    'CL10': ['matsnu', 'nymaim'],
    'CL11': ['locky', 'necurs'],
    'CL12': ['chinad', 'shiotob'],
    'CL13': ['CL06', 'CL07', 'CL08', 'CL11'],
    'CL14': ['CL03', 'vawtrak_v1', 'tinba'],
    'CL15': ['CL09', 'CL13', 'CL14'],
    'CL16': ['CL01', 'CL15', 'kraken'],
}

In [3]:
%%time
data = arff.loadarff(DATA_DIR + "M10K-000.arff")
features = pd.DataFrame(data[0])
#features = pd.read_csv(DATA_DIR + "M10K-PCA.arff", low_memory=False)

features['class'] = features['class'].str.decode('utf-8')
features.dropna(inplace=True)

print('Complete Data Shape:', features.shape)
del(data)

Complete Data Shape: (506810, 131)
CPU times: user 38.2 s, sys: 1.16 s, total: 39.3 s
Wall time: 38.4 s


In [4]:
features['class'].unique()

array(['shiotob', 'gozi_luther', 'bedep', 'tempedreve', 'ramnit',
       'kraken_v2', 'pykspa_noise', 'padcrypt', 'rovnix', 'qakbot',
       'simda', 'corebot', 'banjori', 'vawtrak_v3', 'suppobox_3',
       'ranbyus_v2', 'zeus-newgoz', 'fobber_v2', 'dircrypt', 'suppobox_1',
       'sisron', 'murofet_v3', 'pushdo', 'ccleaner', 'locky',
       'cryptolocker', 'symmi', 'dyre', 'fobber_v1', 'vawtrak_v1',
       'legit', 'alureon', 'gozi_nasa', 'tinba', 'matsnu', 'chinad',
       'proslikefan', 'murofet_v2', 'kraken_v1', 'pizd', 'necurs',
       'gozi_rfc4343', 'murofet_v1', 'qadars', 'pykspa', 'suppobox_2',
       'ranbyus_v1', 'nymaim', 'gozi_gpl', 'vawtrak_v2', 'ramdo'],
      dtype=object)

In [5]:
%%time
# Replace the classes
for key,values in to_be_replaced.items():
    for value in values:
        features['class'] = features['class'].str.replace(pat=value, repl=key, regex=False)
        
# Rebalance the dataset to remove random data from the replaced classes
features = features.sample(frac=1, random_state=42).groupby(by=['class']).head(10000)

CPU times: user 9.06 s, sys: 43.8 ms, total: 9.11 s
Wall time: 9.09 s


In [6]:
features.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209628 entries, 98509 to 122254
Data columns (total 131 columns):
nlp_1g_25p        float64
nlp_1g_50p        float64
nlp_1g_75p        float64
nlp_1g_cov        float64
nlp_1g_dist       float64
nlp_1g_dst_ca     float64
nlp_1g_dst_ch     float64
nlp_1g_dst_em     float64
nlp_1g_dst_eu     float64
nlp_1g_dst_ji     float64
nlp_1g_dst_kl     float64
nlp_1g_dst_ma     float64
nlp_1g_e          float64
nlp_1g_ken        float64
nlp_1g_kur        float64
nlp_1g_mean       float64
nlp_1g_norm       float64
nlp_1g_pea        float64
nlp_1g_pro        float64
nlp_1g_pstd       float64
nlp_1g_pvar       float64
nlp_1g_qmean      float64
nlp_1g_rep        float64
nlp_1g_ske        float64
nlp_1g_spe        float64
nlp_1g_std        float64
nlp_1g_sumsq      float64
nlp_1g_tkur       float64
nlp_1g_tpstd      float64
nlp_1g_tpvar      float64
nlp_1g_tske       float64
nlp_1g_tstd       float64
nlp_1g_tsum       float64
nlp_1g_tsumsq     float64


In [7]:
labels = features['class'].astype('category')
features.drop('class', inplace=True, axis=1) 
display(labels.cat.categories)

Index(['CL02', 'CL04', 'CL05', 'CL10', 'CL12', 'CL16', 'banjori', 'ccleaner',
       'corebot', 'dyre', 'legit', 'murofet_v3', 'padcrypt', 'pushdo', 'ramdo',
       'simda', 'sisron', 'suppobox_2', 'suppobox_3', 'symmi', 'zeus-newgoz'],
      dtype='object')

In [8]:
features_to_keep = ["nlp_l_2dn", 
                    "nlp_r_con_2dn", 
                    "nlp_r_let_fqdn",
                    "nlp_1g_norm",
                    "nlp_1g_tpstd",
                    "nlp_2g_dst_em",
                    "nlp_2g_tpstd",
                    "nlp_3g_dst_em",
                    "nlp_3g_dst_eu",
                    "nlp_3g_norm"
                   ]

features = features[features_to_keep]
features.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209628 entries, 98509 to 122254
Data columns (total 10 columns):
nlp_l_2dn         209628 non-null float64
nlp_r_con_2dn     209628 non-null float64
nlp_r_let_fqdn    209628 non-null float64
nlp_1g_norm       209628 non-null float64
nlp_1g_tpstd      209628 non-null float64
nlp_2g_dst_em     209628 non-null float64
nlp_2g_tpstd      209628 non-null float64
nlp_3g_dst_em     209628 non-null float64
nlp_3g_dst_eu     209628 non-null float64
nlp_3g_norm       209628 non-null float64
dtypes: float64(10)
memory usage: 17.6 MB


In [9]:
category_map = dict( enumerate(labels.cat.categories ) )
for k,v in category_map.items():
    category_map[k] = v.replace("b'","").replace("'","")

category_map_reversed = {}
for k,v in category_map.items():
    category_map_reversed[v] = k
    
with open(DATA_DIR + "category_map-"+str(len(labels.cat.categories))+".labels", 'wb') as dumpfile:
    pickle.dump(category_map, dumpfile)
with open(DATA_DIR + "category_map_reversed-"+str(len(labels.cat.categories))+".labels", 'wb') as dumpfile:
    pickle.dump(category_map_reversed, dumpfile)
    
category_map

{0: 'CL02',
 1: 'CL04',
 2: 'CL05',
 3: 'CL10',
 4: 'CL12',
 5: 'CL16',
 6: 'banjori',
 7: 'ccleaner',
 8: 'corebot',
 9: 'dyre',
 10: 'legit',
 11: 'murofet_v3',
 12: 'padcrypt',
 13: 'pushdo',
 14: 'ramdo',
 15: 'simda',
 16: 'sisron',
 17: 'suppobox_2',
 18: 'suppobox_3',
 19: 'symmi',
 20: 'zeus-newgoz'}

In [10]:
%%time
from sklearn.model_selection import train_test_split

print('Features Shape:', features.shape)

# Split the data into training and testing sets
sampled_features, _, sampled_labels, _ = train_test_split(features, labels.cat.codes, train_size=200000, random_state = 42)

print('Sampled Features Shape:', sampled_features.shape)
print('Sampled Labels Shape:', sampled_labels.shape)

Features Shape: (209628, 10)
Sampled Features Shape: (200000, 10)
Sampled Labels Shape: (200000,)
CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 20.4 ms


In [11]:
sampled_labels.unique()

array([11,  4, 16,  3,  2,  6,  0, 19, 10, 15,  5,  7,  9,  8, 12,  1, 14,
       20, 13, 17, 18])

## Test and train data

In [12]:
%%time
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(sampled_features, sampled_labels, test_size = 0.20, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Training Instances per class:', math.floor(train_labels.shape[0]/len(sampled_labels.unique())))
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)
print('Testing Instances per class:', math.floor(test_labels.shape[0]/len(sampled_labels.unique())))

Training Features Shape: (160000, 10)
Training Labels Shape: (160000,)
Training Instances per class: 7619
Testing Features Shape: (40000, 10)
Testing Labels Shape: (40000,)
Testing Instances per class: 1904
CPU times: user 23.7 ms, sys: 0 ns, total: 23.7 ms
Wall time: 22.5 ms


## Test with LightGBM

In [13]:
%%time
import lightgbm as lgb
model = lgb.LGBMClassifier(n_jobs=4)

times = {}
times['train'] = {
        'wall': {'total': 0, 'instance': 0},
        'user': {'total': 0, 'instance': 0},
    }
times['test'] = {
        'wall': {'total': 0, 'instance': 0},
        'user': {'total': 0, 'instance': 0},
    }

start_time, start_resources = timestamp(), resource_usage(RUSAGE_SELF)
model.fit(train_features, train_labels)
end_resources, end_time = resource_usage(RUSAGE_SELF), timestamp()
times['train']['wall']['total'] = timedelta(seconds=end_time - start_time)
times['train']['wall']['instance'] = times['train']['wall']['total'] / train_features.shape[1]
times['train']['user']['total'] = timedelta(seconds=end_resources.ru_utime - start_resources.ru_utime)
times['train']['user']['instance'] = times['train']['user']['total'] / train_features.shape[1]

start_time, start_resources = timestamp(), resource_usage(RUSAGE_SELF)
model.score(test_features, test_labels)
end_resources, end_time = resource_usage(RUSAGE_SELF), timestamp()
times['test']['wall']['total'] = timedelta(seconds=end_time - start_time)
times['test']['wall']['instance'] = times['test']['wall']['total'] / test_features.shape[1]
times['test']['user']['total'] = timedelta(seconds=end_resources.ru_utime - start_resources.ru_utime)
times['test']['user']['instance'] = times['test']['user']['total'] / test_features.shape[1]

CPU times: user 41.8 s, sys: 380 ms, total: 42.2 s
Wall time: 10.8 s


In [ ]:
%%time
test_labels_predicted = model.predict(test_features)
print("Accuracy:   ", metrics.accuracy_score(test_labels, test_labels_predicted))
print("Prec. Micro:", metrics.precision_score(test_labels, test_labels_predicted, average='micro'))
print("Prec. Macro:", metrics.precision_score(test_labels, test_labels_predicted, average='macro'))
print("F1 Micro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='micro'))
print("F1 Macro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='macro'))

scores = {
    "Accuracy":    metrics.accuracy_score(test_labels, test_labels_predicted),
    "Prec. Micro": metrics.precision_score(test_labels, test_labels_predicted, average='micro'),
    "Prec. Macro": metrics.precision_score(test_labels, test_labels_predicted, average='macro'),
    "F1 Micro":    metrics.f1_score(test_labels, test_labels_predicted, average='micro'),
    "F1 Macro":    metrics.f1_score(test_labels, test_labels_predicted, average='macro'),
}

cfmtrx = metrics.confusion_matrix(test_labels, test_labels_predicted)
fig = plt.figure(figsize=(20,10))
g = sns.heatmap(cfmtrx, yticklabels=labels.cat.categories, xticklabels=labels.cat.categories, annot=False, cmap="Reds", linecolor="#cfcfcf", linewidths=0.01)
bottom, top = g.get_ylim()
g.set_ylim(bottom + 0.5, top - 0.5)
left, right = g.get_xlim()
g.set_xlim(left, right + 0.5)
g.set_title("LightGBM\nFeatures: " + str(train_features.shape[1]) + " - Classes: " + str(len(list(labels.cat.categories))))

text = "Scores\n" + '\n'.join([k+": "+ "{:.3f}".format(v) for k,v in scores.items()]) \
    + "\n\nTraining\nInstances: " + str(train_features.shape[0]) + "\nInstances per class: " + str(math.floor(train_features.shape[0]/len(sampled_labels.unique()))) \
    + "\nUser Time: " + str(times['train']['user']['total']) + "\nUser Time per instance: " + str(times['train']['user']['instance']) \
    + "\nWall Time: " + str(times['train']['wall']['total']) + "\nWall Time per instance: " + str(times['train']['wall']['instance']) \
    + "\n\nTesting\nInstances: " + str(test_features.shape[0]) + "\nInstances per class: " + str(math.floor(test_features.shape[0]/len(sampled_labels.unique()))) \
    + "\nUser Time: " + str(times['test']['user']['total']) + "\nUser Time per instance: " + str(times['test']['user']['instance']) \
    + "\nWall Time: " + str(times['test']['wall']['total']) + "\nWall Time per instance: " + str(times['test']['wall']['instance']) \
    + "\n\nModel Configuration\n" + str(model)
g.text(1.25, 0.95, text, transform=g.transAxes, fontsize=14, verticalalignment='top')
plt.tight_layout()
plt.savefig(GRAPHICS_DIR + "LightGBM-Cl"+ str(len(list(labels.cat.categories))) +"-TrIn"+str(train_features.shape[0])+"." + FIG_EXTENSION)
plt.show()
plt.close()

In [ ]:
import pickle
with open(MODELS_DIR + "lightgbm-Cl"+ str(len(list(labels.cat.categories))) +"-TrIn"+str(train_features.shape[0])+".scikit", 'wb') as dumpfile:
    pickle.dump(model, dumpfile)

## Test with Random Forest

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100, random_state = 42, n_jobs=4)

times = {}
times['train'] = {
        'wall': {'total': 0, 'instance': 0},
        'user': {'total': 0, 'instance': 0},
    }
times['test'] = {
        'wall': {'total': 0, 'instance': 0},
        'user': {'total': 0, 'instance': 0},
    }

start_time, start_resources = timestamp(), resource_usage(RUSAGE_SELF)
model.fit(train_features, train_labels)
end_resources, end_time = resource_usage(RUSAGE_SELF), timestamp()
times['train']['wall']['total'] = timedelta(seconds=end_time - start_time)
times['train']['wall']['instance'] = times['train']['wall']['total'] / train_features.shape[1]
times['train']['user']['total'] = timedelta(seconds=end_resources.ru_utime - start_resources.ru_utime)
times['train']['user']['instance'] = times['train']['user']['total'] / train_features.shape[1]

start_time, start_resources = timestamp(), resource_usage(RUSAGE_SELF)
model.score(test_features, test_labels)
end_resources, end_time = resource_usage(RUSAGE_SELF), timestamp()
times['test']['wall']['total'] = timedelta(seconds=end_time - start_time)
times['test']['wall']['instance'] = times['test']['wall']['total'] / test_features.shape[1]
times['test']['user']['total'] = timedelta(seconds=end_resources.ru_utime - start_resources.ru_utime)
times['test']['user']['instance'] = times['test']['user']['total'] / test_features.shape[1]

In [ ]:
%%time
test_labels_predicted = model.predict(test_features)
print("Accuracy:   ", metrics.accuracy_score(test_labels, test_labels_predicted))
print("Prec. Micro:", metrics.precision_score(test_labels, test_labels_predicted, average='micro'))
print("Prec. Macro:", metrics.precision_score(test_labels, test_labels_predicted, average='macro'))
print("F1 Micro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='micro'))
print("F1 Macro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='macro'))

scores = {
    "Accuracy":    metrics.accuracy_score(test_labels, test_labels_predicted),
    "Prec. Micro": metrics.precision_score(test_labels, test_labels_predicted, average='micro'),
    "Prec. Macro": metrics.precision_score(test_labels, test_labels_predicted, average='macro'),
    "F1 Micro":   metrics.f1_score(test_labels, test_labels_predicted, average='micro'),
    "F1 Macro":   metrics.f1_score(test_labels, test_labels_predicted, average='macro'),
}

cfmtrx = metrics.confusion_matrix(test_labels, test_labels_predicted)
fig = plt.figure(figsize=(20,10))
g = sns.heatmap(cfmtrx, yticklabels=labels.cat.categories, xticklabels=labels.cat.categories, annot=False, cmap="Reds", linecolor="#cfcfcf", linewidths=0.01)
bottom, top = g.get_ylim()
g.set_ylim(bottom + 0.5, top - 0.5)
left, right = g.get_xlim()
g.set_xlim(left, right + 0.5)
g.set_title("Random Forest\nFeatures: " + str(train_features.shape[1]) + " - Classes: " + str(len(list(labels.cat.categories))))

text = "Scores\n" + '\n'.join([k+": "+ "{:.3f}".format(v) for k,v in scores.items()]) \
    + "\n\nTraining\nInstances: " + str(train_features.shape[0]) + "\nInstances per class: " + str(math.floor(train_features.shape[0]/len(sampled_labels.unique()))) \
    + "\nUser Time: " + str(times['train']['user']['total']) + "\nUser Time per instance: " + str(times['train']['user']['instance']) \
    + "\nWall Time: " + str(times['train']['wall']['total']) + "\nWall Time per instance: " + str(times['train']['wall']['instance']) \
    + "\n\nTesting\nInstances: " + str(test_features.shape[0]) + "\nInstances per class: " + str(math.floor(test_features.shape[0]/len(sampled_labels.unique()))) \
    + "\nUser Time: " + str(times['test']['user']['total']) + "\nUser Time per instance: " + str(times['test']['user']['instance']) \
    + "\nWall Time: " + str(times['test']['wall']['total']) + "\nWall Time per instance: " + str(times['test']['wall']['instance']) \
    + "\n\nModel Configuration\n" + str(model)
g.text(1.25, 0.95, text, transform=g.transAxes, fontsize=14, verticalalignment='top')
plt.tight_layout()
plt.savefig(GRAPHICS_DIR + "RandomForest-Cl"+ str(len(list(labels.cat.categories))) +"-TrIn"+str(train_features.shape[0])+"." + FIG_EXTENSION)
plt.show()
plt.close()

In [ ]:
with open(MODELS_DIR + "randomforest-Cl"+ str(len(list(labels.cat.categories))) +"-TrIn"+str(train_features.shape[0])+".scikit", 'wb') as dumpfile:
    pickle.dump(model, dumpfile)

## Test with XGBoost

In [ ]:
%%time
import xgboost as xgb
model = xgb.XGBClassifier(random_state=42, learning_rate=0.01, n_jobs=4)

times = {}
times['train'] = {
        'wall': {'total': 0, 'instance': 0},
        'user': {'total': 0, 'instance': 0},
    }
times['test'] = {
        'wall': {'total': 0, 'instance': 0},
        'user': {'total': 0, 'instance': 0},
    }

start_time, start_resources = timestamp(), resource_usage(RUSAGE_SELF)
model.fit(train_features, train_labels)
end_resources, end_time = resource_usage(RUSAGE_SELF), timestamp()
times['train']['wall']['total'] = timedelta(seconds=end_time - start_time)
times['train']['wall']['instance'] = times['train']['wall']['total'] / train_features.shape[1]
times['train']['user']['total'] = timedelta(seconds=end_resources.ru_utime - start_resources.ru_utime)
times['train']['user']['instance'] = times['train']['user']['total'] / train_features.shape[1]

start_time, start_resources = timestamp(), resource_usage(RUSAGE_SELF)
model.score(test_features, test_labels)
end_resources, end_time = resource_usage(RUSAGE_SELF), timestamp()
times['test']['wall']['total'] = timedelta(seconds=end_time - start_time)
times['test']['wall']['instance'] = times['test']['wall']['total'] / test_features.shape[1]
times['test']['user']['total'] = timedelta(seconds=end_resources.ru_utime - start_resources.ru_utime)
times['test']['user']['instance'] = times['test']['user']['total'] / test_features.shape[1]

In [ ]:
%%time
test_labels_predicted = model.predict(test_features)
print("Accuracy:   ", metrics.accuracy_score(test_labels, test_labels_predicted))
print("Prec. Micro:", metrics.precision_score(test_labels, test_labels_predicted, average='micro'))
print("Prec. Macro:", metrics.precision_score(test_labels, test_labels_predicted, average='macro'))
print("F1 Micro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='micro'))
print("F1 Macro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='macro'))


scores = {
    "Accuracy":    metrics.accuracy_score(test_labels, test_labels_predicted),
    "Prec. Micro": metrics.precision_score(test_labels, test_labels_predicted, average='micro'),
    "Prec. Macro": metrics.precision_score(test_labels, test_labels_predicted, average='macro'),
    "F1 Micro":   metrics.f1_score(test_labels, test_labels_predicted, average='micro'),
    "F1 Macro":   metrics.f1_score(test_labels, test_labels_predicted, average='macro'),
}

cfmtrx = metrics.confusion_matrix(test_labels, test_labels_predicted)
fig = plt.figure(figsize=(20,10))
g = sns.heatmap(cfmtrx, yticklabels=labels.cat.categories, xticklabels=labels.cat.categories, annot=False, cmap="Reds", linecolor="#cfcfcf", linewidths=0.01)
bottom, top = g.get_ylim()
g.set_ylim(bottom + 0.5, top - 0.5)
left, right = g.get_xlim()
g.set_xlim(left, right + 0.5)
g.set_title("XGBoost\nFeatures: " + str(train_features.shape[1]) + " - Classes: " + str(len(list(labels.cat.categories))))

text = "Scores\n" + '\n'.join([k+": "+ "{:.3f}".format(v) for k,v in scores.items()]) \
    + "\n\nTraining\nInstances: " + str(train_features.shape[0]) + "\nInstances per class: " + str(math.floor(train_features.shape[0]/len(sampled_labels.unique()))) \
    + "\nUser Time: " + str(times['train']['user']['total']) + "\nUser Time per instance: " + str(times['train']['user']['instance']) \
    + "\nWall Time: " + str(times['train']['wall']['total']) + "\nWall Time per instance: " + str(times['train']['wall']['instance']) \
    + "\n\nTesting\nInstances: " + str(test_features.shape[0]) + "\nInstances per class: " + str(math.floor(test_features.shape[0]/len(sampled_labels.unique()))) \
    + "\nUser Time: " + str(times['test']['user']['total']) + "\nUser Time per instance: " + str(times['test']['user']['instance']) \
    + "\nWall Time: " + str(times['test']['wall']['total']) + "\nWall Time per instance: " + str(times['test']['wall']['instance']) \
    + "\n\nModel Configuration\n" + str(model)
g.text(1.25, 0.95, text, transform=g.transAxes, fontsize=14, verticalalignment='top')
plt.tight_layout()
plt.savefig(GRAPHICS_DIR + "XGBClassifier-Cl"+ str(len(list(labels.cat.categories))) +"-TrIn"+str(train_features.shape[0])+"." + FIG_EXTENSION)
plt.show()
plt.close()

In [ ]:
with open(MODELS_DIR + "xgboost-Cl"+ str(len(list(labels.cat.categories))) +"-TrIn"+str(train_features.shape[0])+".scikit", 'wb') as dumpfile:
    pickle.dump(model, dumpfile)

### GridSearch

In [ ]:
%%time
parameters = {
    'num_leaves': [len(labels.unique()), 50, 75, 85, 100],
    'n_estimators': [50, 75, 85, 100, 120, 150, 200],
}
search = model_selection.GridSearchCV(model, param_grid=parameters, cv=5, n_jobs=1, pre_dispatch=1)

In [ ]:
%%time
search.fit(features, labels)

In [ ]:
res = pd.DataFrame(search.cv_results_)
res

In [ ]:
res.param_max_depth.fillna(value="Unlim", inplace=True)
res.param_max_features.fillna(value="Auto", inplace=True)
#res.param_n_estimators = "p_" + res.param_n_estimators.astype(str)
res.sample(5)

In [ ]:
res.info()

In [ ]:
fig = plt.figure(figsize=(15,10))
g = sns.scatterplot(data=res, 
                    x=res.mean_fit_time, 
                    y=res.mean_test_score, 
                    style=res.param_n_estimators, 
                    #hue=res.param_num_leaves,
                    s=100
                   )

plt.ylim(bottom=0.8, top=1.0)


def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.15, point['y']+.005, str(point['val']))

#label_point(res.mean_fit_time, res.mean_test_score, res.rank_test_score, g) 

plt.show()

In [ ]:
sns.pairplot(data=res, 
             x_vars=['mean_fit_time', 'mean_test_score', 'param_num_leaves'], 
             y_vars=['mean_fit_time', 'mean_test_score', 'param_num_leaves'], 
             hue="param_n_estimators"
             )

In [ ]:
rnd.es

# Get online features

In [ ]:
def process(domain):
    import requests,json,psutil,datetime
    headers = {'Content-Type': 'application/json'}
    try:
        start = datetime.datetime.now()
        r = requests.post("http://localhost:8080/DGA/domain/features", data=json.dumps({"fqdn": domain}), headers=headers)
        #print(r.json())
        features = {}
        for elem in r.json():
            for key in elem.keys():
                features[key] = elem[key]
                
        del(features['class'])
        del(features['domain'])
        return {
            'domain': domain,
            'client_time': (datetime.datetime.now()-start).total_seconds() *1000,
            'status_code': r.status_code,
            'server_time': r.headers['time'],
            'features': features
        }
    except Exception as ex:
        return {
            'domain': domain,
            'exception': ex
        }
    
def classify(domain):
    evaluation_features = pd.DataFrame(columns=train_features.columns)
    evaluation_features = evaluation_features.append(process(domain)['features'], ignore_index=True).astype('float')
    return model.predict(evaluation_features)[0]

In [ ]:
model.classes_

In [ ]:
category_map_reversed

In [ ]:
evaluation_domains = pd.read_csv(DATA_DIR + "Evaluation.csv").sample(frac=1, random_state=42).groupby(by=['label']).head(10)
for key,values in to_be_replaced.items():
    for value in values:
        evaluation_domains['label'] = evaluation_domains['label'].str.replace(pat=value, repl=key, regex=False)
evaluation_domains['predicted'] = None
display(evaluation_domains.info())
display(evaluation_domains.sample())
display(evaluation_domains.label.unique())
evaluation_domains['label_codes'] = np.vectorize(lambda x: category_map_reversed[x])(evaluation_domains['label'])

In [ ]:
%%time
times = {}
times['test'] = {
        'wall': {'total': 0, 'instance': 0},
        'user': {'total': 0, 'instance': 0},
    }

start_time, start_resources = timestamp(), resource_usage(RUSAGE_SELF)
evaluation_domains['predicted'] = np.vectorize(lambda domain: category_map[classify(domain)])(evaluation_domains['domain'])
end_resources, end_time = resource_usage(RUSAGE_SELF), timestamp()
times['test']['wall']['total'] = timedelta(seconds=end_time - start_time)
times['test']['wall']['instance'] = times['test']['wall']['total'] / evaluation_domains.shape[0]
times['test']['user']['total'] = timedelta(seconds=end_resources.ru_utime - start_resources.ru_utime)
times['test']['user']['instance'] = times['test']['user']['total'] / evaluation_domains.shape[0]

In [ ]:
evaluation_domains.predicted.isnull().any()

In [ ]:
%%time
evaluation_domains['predicted_codes'] = np.vectorize(lambda x: category_map_reversed[x])(evaluation_domains['predicted'])
evaluation_domains.sample(10)

In [ ]:
%%time

print("Accuracy:   ", metrics.accuracy_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes']))
print("Prec. Micro:", metrics.precision_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='micro'))
print("Prec. Macro:", metrics.precision_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='macro'))
print("F1 Micro:   ", metrics.f1_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='micro'))
print("F1 Macro:   ", metrics.f1_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='macro'))


scores = {
    "Accuracy":    metrics.accuracy_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes']),
    "Prec. Micro": metrics.precision_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='micro'),
    "Prec. Macro": metrics.precision_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='macro'),
    "F1 Micro":   metrics.f1_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='micro'),
    "F1 Macro":   metrics.f1_score(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'], average='macro'),
}

cfmtrx = metrics.confusion_matrix(evaluation_domains['label_codes'], evaluation_domains['predicted_codes'])
fig = plt.figure(figsize=(20,10))
g = sns.heatmap(cfmtrx, yticklabels=labels.cat.categories, xticklabels=labels.cat.categories, annot=False, cmap="Reds", linecolor="#cfcfcf", linewidths=0.01)
bottom, top = g.get_ylim()
g.set_ylim(bottom + 0.5, top - 0.5)
left, right = g.get_xlim()
g.set_xlim(left, right + 0.5)
g.set_title("Online Test\nFeatures: " + str(train_features.shape[1]) + " - Classes: " + str(len(list(labels.cat.categories))))

text = "Scores\n" + '\n'.join([k+": "+ "{:.3f}".format(v) for k,v in scores.items()]) \
    + "\n\nTesting\nInstances: " + str(evaluation_domains.shape[0]) + "\nInstances per class: " + str(math.floor(evaluation_domains.shape[0]/len(sampled_labels.unique()))) \
    + "\nUser Time: " + str(times['test']['user']['total']) + "\nUser Time per instance: " + str(times['test']['user']['instance']) \
    + "\nWall Time: " + str(times['test']['wall']['total']) + "\nWall Time per instance: " + str(times['test']['wall']['instance']) \
    + "\n\nModel Configuration\n" + str(model)
g.text(1.25, 0.95, text, transform=g.transAxes, fontsize=14, verticalalignment='top')
plt.tight_layout()
plt.savefig(GRAPHICS_DIR + "OnlineTest." + FIG_EXTENSION)
plt.show()
plt.close()

## Feature Selection

In [ ]:
from sklearn.feature_selection import RFE

import lightgbm as lgb
model = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=20, n_jobs=4)

selector = RFE(model)

In [ ]:
%%time
selector = selector.fit(train_features, train_labels)

In [ ]:
display(selector.support_)
sum(selector.support_)

In [ ]:
features.columns[selector.support_]

In [ ]:
selector.ranking_

In [ ]:
model = selector.estimator_

In [ ]:
reduced_test_features = selector.transform(test_features)
model.score(reduced_test_features, test_labels)

In [ ]:
%%time
test_labels_predicted = model.predict(reduced_test_features)
print("Accuracy:   ", metrics.accuracy_score(test_labels, test_labels_predicted))
print("Prec. Micro:", metrics.precision_score(test_labels, test_labels_predicted, average='micro'))
print("Prec. Macro:", metrics.precision_score(test_labels, test_labels_predicted, average='macro'))
print("F1 Micro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='micro'))
print("F1 Macro:   ", metrics.f1_score(test_labels, test_labels_predicted, average='macro'))

cfmtrx = metrics.confusion_matrix(test_labels, test_labels_predicted)
fig = plt.figure(figsize=(12,10))
g = sns.heatmap(cfmtrx, yticklabels=labels.cat.categories, xticklabels=labels.cat.categories, annot=False, cmap="Reds", linecolor="#cfcfcf", linewidths=0.01)
bottom, top = g.get_ylim()
g.set_ylim(bottom + 0.5, top - 0.5)
left, right = g.get_xlim()
g.set_xlim(left, right + 0.5)

## Autoencoder

In [ ]:
def test_autoencoder(features, labels, enc_layers, dec_layers, layers_text="", epochs=32, batch_size=1000):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler    
    from keras.models import Model
    scaler = MinMaxScaler()
    features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
    features.describe()
    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels.cat.codes, test_size = 0.20, random_state = 42)

    input_layer = Input(shape = (train_features.shape[1], ), name="I")
    output_layer = input_layer

    for layer in enc_layers:
        output_layer = layer(output_layer)

    for layer in dec_layers:
        output_layer = layer(output_layer)

    autoencoder = Model(input = input_layer, output = output_layer, name="Autoencoder")

    # CONFIGURE AND TRAIN THE AUTOENCODER
    autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')
    autoencoder.summary()

    autoencoder.fit(train_features, train_features, epochs = epochs, batch_size = batch_size, shuffle = True, validation_data = (test_features, test_features), workers=4)

    test = autoencoder.get_layer("E_OUT")(autoencoder.get_layer("E3")(autoencoder.get_layer("E2")(autoencoder.get_layer("E1")(input_layer))))
    
    encoder_model = Model(input = input_layer, output = test, name="Encoder")
    encoded_input = Input(shape = (encoding_dim, ))
    encoded_train_features = encoder_model.predict(train_features)
    encoded_test_features = encoder_model.predict(test_features)
    
    import lightgbm as lgb
    model = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=20, n_jobs=4)
    model.fit(encoded_train_features, train_labels)
    model.score(encoded_test_features, test_labels)

    test_labels_predicted = model.predict(encoded_test_features)
    
    scores = {
        "Accuracy":    metrics.accuracy_score(test_labels, test_labels_predicted),
        "Prec. Micro": metrics.precision_score(test_labels, test_labels_predicted, average='micro'),
        "Prec. Macro": metrics.precision_score(test_labels, test_labels_predicted, average='macro'),
        "F1 Micro":   metrics.f1_score(test_labels, test_labels_predicted, average='micro'),
        "F1 Macro":   metrics.f1_score(test_labels, test_labels_predicted, average='macro'),
    }
    
    cfmtrx = metrics.confusion_matrix(test_labels, test_labels_predicted)
    fig = plt.figure(figsize=(12,10))
    g = sns.heatmap(cfmtrx, yticklabels=labels.cat.categories, xticklabels=labels.cat.categories, annot=False, cmap="Reds", linecolor="#cfcfcf", linewidths=0.01)
    bottom, top = g.get_ylim()
    g.set_ylim(bottom + 0.5, top - 0.5)
    left, right = g.get_xlim()
    g.set_xlim(left, right + 0.5)
    g.set_title("LightGBM + Autoencoder (" + str(encoded_train_features.shape[1]) + "neurons)")


    text = "Scores\n" + '\n'.join([k+": "+ "{:.3f}".format(v) for k,v in scores.items()]) + "\n\nAutoencoder Model:\n" + layers_text
    g.text(0.55, 0.95, text, transform=g.transAxes, fontsize=14, verticalalignment='top')

    plt.show()
    plt.close()

    return scores

In [ ]:
from keras.layers import Input, Dense

layers = ""

# DEFINE THE DIMENSION OF ENCODER
encoding_dim = 4*len(labels.unique())

# DEFINE THE ENCODER LAYERS
enc_layers = []
layers += "Input("+str(train_features.shape[1])+")\n"
enc_layers.append(Dense(256, activation = 'relu', name="E1"))
layers += "E1: Dense(256, 'relu')\n"
enc_layers.append(Dense(512, activation = 'relu', name="E2"))
layers += "E2: Dense(128, 'relu')\n"
enc_layers.append(Dense(1024, activation = 'relu', name="E3"))
layers += "E3: Dense(64, 'relu')\n"
enc_layers.append(Dense(encoding_dim, activation = 'relu', name="E_OUT"))
layers += "E_OUT: Dense("+str(encoding_dim)+", 'relu')\n"
layers += "---------------\n"

dec_layers = []
# DEFINE THE DECODER LAYERS
enc_layers.append(Dense(1024, activation = 'relu', name="D1"))
layers += "D1: Dense(64, 'relu')\n"
enc_layers.append(Dense(512, activation = 'relu', name="D2"))
layers += "D2: Dense(128, 'relu')\n"
enc_layers.append(Dense(256, activation = 'relu', name="D3"))
layers += "D3: Dense(256, 'relu')\n"
enc_layers.append(Dense(train_features.shape[1], activation = 'sigmoid', name="D_OUT"))
layers += "D_OUT: Dense("+str(train_features.shape[1])+", 'sigmoid')\n"
    
test_autoencoder(features, labels, enc_layers, dec_layers, layers_text = layers, epochs=32, batch_size=1000)